In [1]:
# Standard library imports
import numpy as np
import matplotlib.pyplot as plt

# Third-party specialized imports
import tifffile
import napari

# Scikit-image imports
from skimage import data, filters, io
from skimage.measure import label, regionprops
from skimage.morphology import (
    disk, 
    dilation, 
    remove_small_objects, 
    skeletonize
)
from skimage.segmentation import expand_labels, find_boundaries

from scipy import ndimage

# Jupyter notebook integration
%gui qt



In [2]:
# All variable parameters are here
input_file = 'data/yeast.tif'
ground_truth_file = 'data/cellpose_truth.tif'
show_images = True
save_result = False

In [3]:
if show_images:
    # Create a viewer
    viewer = napari.Viewer()
    print("Setup successful!")

Setup successful!


In [4]:
image = io.imread(input_file)
plane_of_interest = image[1,:,:] # Extract second plane

if show_images:
    viewer.add_image(plane_of_interest, name='Original', colormap='gray')


# Display cellpose results, considered ground truth
cellpose_mask = io.imread('data/cellpose_mask.tif')

if show_images:
    viewer.add_labels(cellpose_mask, name="Cellpose Label Image")


In [5]:
import pandas as pd
import numpy as np

# Create empty DataFrame with all columns
columns = [
    'blur_radius_for_normalisation',
    'median_filter_size', 
    'threshold_bright_pixels',
    'remove_objects_below_size',
    'min_distance_bridge_cut',
    'dilate_borders_px',
    'remove_objects_above_size',
    'label_expansion',
    'IoU_Score',
    'Ground_Truth_Objects',
    'Detected_Objects'
]

results_table = pd.DataFrame(columns=columns)
print("Results table initialized:")

from IPython.display import display
display(results_table)


Results table initialized:


,blur_radius_for_normalisation,median_filter_size,threshold_bright_pixels,remove_objects_below_size,min_distance_bridge_cut,dilate_borders_px,remove_objects_above_size,label_expansion,IoU_Score,Ground_Truth_Objects,Detected_Objects


In [6]:
# Analysis parameters

# Pseudo flat field: we divide the image by its gaussian blurred version
blur_radius_for_normalisation = 20

# Removes noise from the image
median_filter_size = 3

# Threshold bright pixels which are surrounding the cell
threshold_bright_pixels = 1.14 # 1.15 original

# Remove small objects which are assumed not to represent any cell border
remove_objects_below_size = 140 # 150 original

# Splits seeds connected by a thin bridge
min_distance_bridge_cut = 4

# Dilate border mask. Goal: closing cells, without losing small ones. Value too big: small cell are missed. Values too small: cells remain opened, thus closed
dilate_borders_px = 9 # original 10

# Once the image is becoming a label image, we remove the labels which are too big: they are not cells but rather the background or interstices between cells
remove_objects_above_size = 5000

# Makes cells more blobby, make them match better their outline
label_expansion = 16

In [7]:
# Normalize: divide by the gaussian blurred version of the image (radius 20 pixels), output is 32 bits
def normalize_with_blur(image, blur_radius):
    """
    Normalize image by dividing by its Gaussian blurred version.
    Returns 32-bit float result.
    """
    # Gaussian blur
    blurred = filters.gaussian(image.astype(np.float32), sigma=blur_radius)
    
    # Normalize and convert to 32-bit float
    normalized = (image.astype(np.float32) / blurred).astype(np.float32)
    
    return normalized

In [8]:
normalized_image = normalize_with_blur(plane_of_interest, blur_radius=blur_radius_for_normalisation)

if show_images:
    viewer.add_image(normalized_image, name='Normalized', colormap='gray')

In [9]:
# Apply median filter
filtered_image = filters.median(normalized_image, disk(median_filter_size))

if show_images:
    viewer.add_image(filtered_image, name='Median_Disk_3', colormap='gray')

In [10]:
# Keeps bright pixels surrounding cells
thresholded_image = (filtered_image>threshold_bright_pixels)

if show_images:
    viewer.add_image(thresholded_image, name='Threshold_'+str(threshold_bright_pixels), colormap='gray')

In [11]:
# Remove small connected components, 8-bits connectivity
cleaned_image = remove_small_objects(thresholded_image, min_size=remove_objects_below_size, connectivity=2)

if show_images:
    viewer.add_image(cleaned_image, name='Cleaned Objects below '+str(remove_objects_below_size), colormap='gray')

In [12]:
# Dilate border image

dilated_image = dilation(cleaned_image, disk(dilate_borders_px))

if show_images:
    viewer.add_image(dilated_image, name='Dilated_'+str(dilate_borders_px), colormap='gray')

In [13]:
# Keep only objects SMALLER than a certain size
def keep_small_objects(binary_mask, max_size):
    # Remove objects >= max_size (keeping smaller ones)
    large_objects = remove_small_objects(binary_mask, min_size=max_size)
    # Subtract large objects from original to keep only small ones
    small_objects_only = binary_mask & ~large_objects
    return small_objects_only

# Usage
seeds_raw = keep_small_objects(~dilated_image, max_size=remove_objects_above_size)

if show_images:
    viewer.add_image(seeds_raw, name='Seeds', colormap='gray')


In [14]:

def distance_threshold_mask(binary_mask, min_distance=2):
    """
    Create a mask keeping only pixels that are at least min_distance 
    away from the edge of objects.
    """
    # Compute distance transform (distance to nearest background pixel)
    distance = ndimage.distance_transform_edt(binary_mask)
    
    # Keep only pixels with distance >= min_distance
    thresholded_mask = distance >= min_distance
    
    return thresholded_mask

seeds_split = distance_threshold_mask(seeds_raw, min_distance=min_distance_bridge_cut)

if show_images:
    viewer.add_image(seeds_split, name='Seeds Split', colormap='gray')

In [15]:
# Now let's invert and mark as label

# Convert binary image to labeled image
labels_raw = label(seeds_split)

if show_images:
    viewer.add_labels(labels_raw, name = "Seeds Labels")

In [16]:
# Expand labels to cover the whole cell
labels = expand_labels(labels_raw, distance=label_expansion)

if show_images:
    # Find boundaries between labeled regions
    viewer.add_labels(labels, name="Labels, Final")

if save_result:
    # save results to tif
    import tifffile
    
    # Save the labeled mask as TIFF
    tifffile.imwrite('data/result_mask_v3.tif', labels)

if show_images:
    # Find boundaries between labeled regions
    boundaries = find_boundaries(labels, mode='thick')
    
    # Create a display image: labels with black boundaries
    blobby_labels_eroded = labels.copy()
    blobby_labels_eroded[boundaries] = 0  # Set boundaries to 0 (black)
    
    viewer.add_labels(boundaries, name="Cell Delimitation")

In [17]:
import numpy as np

def iou_binary(labels1, labels2):
    """Compute IoU between two label images (converted to binary)."""
    # Convert to binary (any label > 0)
    binary1 = labels1 > 0
    binary2 = labels2 > 0
    
    # Compute intersection and union
    intersection = np.sum(binary1 & binary2)
    union = np.sum(binary1 | binary2)
    
    # Avoid division by zero
    if union == 0:
        return 1.0 if intersection == 0 else 0.0
    
    iou = intersection / union
    return iou

# Usage
iou_score = iou_binary(labels, cellpose_mask)
print(f"IoU: {iou_score:.3f}")

IoU: 0.691


In [18]:
# Calculate metrics
iou_score = iou_binary(labels, cellpose_mask)
n_gt_objects = len(np.unique(cellpose_mask)) - 1
n_detected_objects = len(np.unique(labels)) - 1


# Create new row with current parameters and results
new_row = {
    'blur_radius_for_normalisation': blur_radius_for_normalisation,
    'median_filter_size': median_filter_size,
    'threshold_bright_pixels': threshold_bright_pixels,
    'remove_objects_below_size': remove_objects_below_size,
    'min_distance_bridge_cut': min_distance_bridge_cut,
    'dilate_borders_px': dilate_borders_px,
    'remove_objects_above_size': remove_objects_above_size,
    'label_expansion': label_expansion,
    'IoU_Score': round(iou_score, 3),
    'Ground_Truth_Objects': n_gt_objects,
    'Detected_Objects': n_detected_objects
}

# Add to results table
results_table = pd.concat([results_table, pd.DataFrame([new_row])], ignore_index=True)

# Display updated table
print(f"Experiment {len(results_table)} added:")
display(results_table)

Experiment 1 added:


C:\Users\Nicolas\AppData\Local\Temp\ipykernel_13980\3840769688.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_table = pd.concat([results_table, pd.DataFrame([new_row])], ignore_index=True)


,blur_radius_for_normalisation,median_filter_size,threshold_bright_pixels,remove_objects_below_size,min_distance_bridge_cut,dilate_borders_px,remove_objects_above_size,label_expansion,IoU_Score,Ground_Truth_Objects,Detected_Objects
0,20,3,1.14,140,4,9,5000,16,0.691,501,421
